In [1]:
#import pandas
#df = pandas.read_csv('data/biographynet/.csv')
#print(df)


import pandas as pd
import glob
import re

path_bio = r'data/biographynet'
path_euro_ca = r'data/europeana-ca'
path_euro_qm = r'data/europeana-?'
all_files = glob.glob(path_bio + "/*.csv")
all_euro_ca_files = glob.glob(path_euro_ca + "/*.csv")
all_euro_qm_files = glob.glob(path_euro_qm + "/*.csv")

li = []
li_euro_ca = []
li_euro_qm = []

for filename in all_files:
    print(filename)
    df = pd.read_csv(filename)
    li.append(df)

for filename in all_euro_ca_files:
    print(filename)
    df = pd.read_csv(filename)
    li_euro_ca.append(df)
    
for filename in all_euro_qm_files:
    print(filename)
    df = pd.read_csv(filename)
    li_euro_qm.append(df)
    
bio = pd.concat(li, axis=0, ignore_index=True)
bio.drop(bio.columns[[0,2]], axis=1, inplace=True)
bio_list = bio.values.tolist()

euro_ca = pd.concat(li_euro_ca, axis=0, ignore_index=True)
euro_ca.drop(euro_ca.columns[[0,1,2]], axis=1, inplace=True)
euro_ca_list = euro_ca.values.tolist()

euro_qm = pd.concat(li_euro_qm, axis=0, ignore_index=True)
euro_qm.drop(euro_qm.columns[[0,1,2]], axis=1, inplace=True)
euro_qm_list = euro_qm.values.tolist()

euro_list = euro_ca_list + euro_qm_list
#print(euro_ca_list)
print(len(euro_qm_list))
print(len(euro_ca_list))
print(len(euro_list))
print(len(bio_list))

print("sum: ",len(euro_ca_list)+len(euro_qm_list)+len(bio_list))

data/biographynet/dash.csv
data/biographynet/na.csv
data/biographynet/questionmark.csv
data/biographynet/voor.csv
data/biographynet/ca.csv
data/europeana-ca/5.csv
data/europeana-ca/2.csv
data/europeana-ca/4.csv
data/europeana-ca/3.csv
data/europeana-ca/1.csv
data/europeana-ca/6.csv
data/europeana-?/2.csv
data/europeana-?/3.csv
data/europeana-?/1.csv
29524
52780
82304
15418
sum:  97722


In [2]:
# CASE: ca. <date>     =>    vague point in time
HAS_CA_AND_YEAR = ".*(ca|\d{3,4}).*(ca|\d{3,4}).*"
NO_DOUBLE_CA_OR_YEAR = "^(?!.*(ca.*?(ca)))(?!.*(\d{4}.*?(\d{4}))).*$"
MATCH_YMD_FORMAT= ".*(\d{4}-\d{2}-\d{2}).*(Ca|ca).*"
YEAR_QM = ".*\d{4}\?.*"


def getCaYearList(date_list):
    ca_year = []
    no_doubles = []
    discard = []
    
    for date in date_list:   
        if re.search(HAS_CA_AND_YEAR, date[0]) and (len(date[0])<25):
            ca_year.append(date[0])
        else:
            if re.search(MATCH_YMD_FORMAT, date[0]):
                ca_year.append(date[0])
            else:
                discard.append(date[0])
            
    #print("discarded: ",len(discard), discard)
    return list(filter(lambda date: re.search(NO_DOUBLE_CA_OR_YEAR, date), ca_year))

#print("list after regex: ",len(getCaYearList(euro_ca_list)),getCaYearList(euro_ca_list))


print(len(getCaYearList(bio_list)))
print(len(getCaYearList(euro_list)))

#print(len(bio_list))
#print(len(getCaYearList(bio_list)))
#print(len(euro_ca_list))
#print(len(getCaYearList(euro_ca_list)))
#print(bio_list)

5007
16047


In [3]:
# CASE: TIMESPANS
# e.g. 
# ca. <YEAR> - <YEAR>
# <YEAR>-?-?       #precision: < 1 year
# X-th century

CA_YEAR_DASH_YEAR = ".*(Ca|ca).*\d{4}.*-.*\d{4}.*" # 1955 [ca] -1960 [ca] not considered on purpose (to not include ca. 1999/ca. 1999)
YEAR_QM_QM = ".*\d{4}-\?-\?.*"                        # only 4 matches
CENTURY = ".*(de eeuw|century|Century|th|jahrhundert|Jahrhundert).*"



def getListWithMatchedDates(regex,dates_to_check):
    discarded = []
    matched = []
    
    for date in dates_to_check:
        if re.search(regex, date[0]):
            matched.append(date[0])
        else:
            discarded.append(date[0])
            
    #print("discarded: ", len(discarded), discarded)
    #print("matched: ", len(matched), matched)
    return matched

#print(bio_list)
#euro_year_dash_year = 
print(len(getListWithMatchedDates(CA_YEAR_DASH_YEAR, euro_list)))
print(len(getListWithMatchedDates(YEAR_QM_QM, euro_list)))
print(len(getListWithMatchedDates(CENTURY, euro_list)))
#print(len(euro_year_dash_year))
#print(euro_year_dash_year)

3609
0
1744


In [4]:
# CASE: COMPLETE VAGUENESS
QM = "^[^a-z0-9]*\?[^a-z0-9]*$"  # just a questionmark


print(len(getListWithMatchedDates(QM, euro_list)))

1033


In [5]:
# CASE: BEFORE/AFTER <DATE>

BEFORE_AFTER = ".*(before|after).*\d{4}.*"
print(len(getListWithMatchedDates(BEFORE_AFTER, euro_list)))

33


In [6]:

# CASE: DATE OPTION
# e.g.: 1880/1885
# bei bio_list nur wenige matches

#print(bio_list)
YEAR_OR_YEAR = ".*\d{4}.*\/.*\d{4}.*"
print(len(getListWithMatchedDates(YEAR_OR_YEAR, bio_list)))
# test lololol

5
